NOMBRE:


CARNE:


FECHA:

Responda las siguientes preguntas:
1. ¿Cuáles son los dos procesos principales de un transformer y qué función cumplen?
  

- A. Codificación (Encoder): El proceso de codificación toma la entrada y la convierte en una representación de alto nivel (un conjunto de características), conservando relaciones entre las palabras gracias al mecanismo de atención.

- B. Decodificación (Decoder): El proceso de decodificación toma la representación codificada y la convierte en la salida deseada (por ejemplo, una traducción). También utiliza atención, tanto hacia la entrada codificada como hacia la propia salida parcial.


  
2. Asigne cada uno de las siguientes subtareas a su proceso correspondiente(A o B): Multi-Head Self-Attention Mechanism,Position-wise Feed-Forward Networks,Masked Multi-Head Self-Attention Mechanism, Encoder-Decoder Multi-Head Attention,Position-wise Feed-Forward Networks

Después de asignar, explique la función de cada una de las partes. Luego, proceda a programar un transformer con cada una de esas partes utilizando solo tensorflow o Pytorch.


**Asignación de las subtareas a su proceso correspondiente:**

- Multi-Head Self-Attention Mechanism: Se encuentra en ambos procesos (Encoder y Decoder) y permite que el modelo se enfoque en diferentes partes de la entrada o la salida de manera simultánea.

- Position-wise Feed-Forward Networks: También está presente en ambos (Encoder y Decoder). Se aplica después del mecanismo de atención para capturar patrones no lineales.

- Masked Multi-Head Self-Attention Mechanism: Se encuentra solo en el Decodificador. Enmascara futuras posiciones para evitar que el modelo vea respuestas futuras durante el entrenamiento.

- Encoder-Decoder Multi-Head Attention: Solo está presente en el Decodificador y permite que el decodificador se centre en partes relevantes de la entrada.

**Explicación de las partes:**

- Multi-Head Self-Attention: El modelo puede prestar atención a múltiples partes de la entrada al mismo tiempo, lo que ayuda a capturar relaciones complejas.

- Position-wise Feed-Forward Networks: Se aplica de manera independiente en cada posición y capa para capturar interacciones no lineales entre las características.

- Masked Multi-Head Self-Attention: Evita que el decodificador vea respuestas futuras durante el entrenamiento, asegurando una predicción secuencial.

- Encoder-Decoder Multi-Head Attention: Permite que el decodificador use la información de la representación generada por el codificador para generar la salida.

In [21]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, LayerNormalization

def positional_encoding(position, d_model):
    angle_rads = np.arange(position)[:, np.newaxis] / np.power(10000, (2 * (np.arange(d_model) // 2)) / np.float32(d_model))
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    return tf.cast(angle_rads[np.newaxis, ...], dtype=tf.float32)

Responda: ¿Qué es el posicional encoding y por qué debe definirse una función para los transformer?

In [22]:

# Definimos el TransformerBlock
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(d_model, num_heads)
        self.ffn = self.point_wise_feed_forward_network(d_model, dff)
        
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)
    
    def call(self, x, mask, training=False):
        attn_output, _ = self.att(x, x, x, mask)  # Multi-head attention
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)  # Residual connection + normalization
        
        ffn_output = self.ffn(out1)  # Feed-forward network
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)  # Residual connection + normalization
        return out2
    
    def point_wise_feed_forward_network(self, d_model, dff):
        return tf.keras.Sequential([
            Dense(dff, activation='relu'),  # First layer of FFN
            Dense(d_model)  # Second layer of FFN
        ])

In [23]:
# Multi-Head Attention class as defined before
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        
        assert d_model % self.num_heads == 0
        
        self.depth = d_model // self.num_heads
        self.wq = Dense(d_model)
        self.wk = Dense(d_model)
        self.wv = Dense(d_model)
        self.dense = Dense(d_model)
    
    def split_heads(self, x, batch_size):
        """Divide la última dimensión en múltiples cabezas."""
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])
    
    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]
        
        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)
        
        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)
        
        scaled_attention, attention_weights = self.scaled_dot_product_attention(q, k, v, mask)
        
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))
        
        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)
        return output, attention_weights

    def scaled_dot_product_attention(self, q, k, v, mask):
        """Calcula la atención escalarizada."""
        matmul_qk = tf.matmul(q, k, transpose_b=True)
        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
        
        if mask is not None:
            scaled_attention_logits += (mask * -1e9)
        
        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
        output = tf.matmul(attention_weights, v)
        return output, attention_weights

In [24]:
# Clase completa del Transformer
class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, target_vocab_size, pe_input, pe_target, rate=0.1):
        super(Transformer, self).__init__()
        
        self.encoder = [TransformerBlock(d_model, num_heads, dff, rate) for _ in range(num_layers)]
        self.final_layer = Dense(target_vocab_size)  # Proyección al vocabulario de salida

    def call(self, x, mask=None, training=False):
        for encoder_layer in self.encoder:
            x = encoder_layer(x, mask, training=training)  # Pasamos por las capas del encoder
        
        # Proyectar la salida final al tamaño del vocabulario
        final_output = self.final_layer(x)  # (batch_size, seq_len, target_vocab_size)
        return final_output

In [25]:
#Datos de prueba
num_layers = 4
d_model = 128
num_heads = 8
dff = 512
input_vocab_size = 8500
target_vocab_size = 8000
pe_input = 1000
pe_target = 1000
dropout_rate = 0.1

#Su output shape debe ser: (64, 50, 8000)

In [26]:
# Crear una entrada de prueba
sample_input = tf.random.uniform((64, 50, d_model))  # (batch_size, seq_len, d_model)

# Crear el transformer completo y obtener la salida
transformer = Transformer(num_layers, d_model, num_heads, dff, input_vocab_size, target_vocab_size, pe_input, pe_target, dropout_rate)
sample_output = transformer(sample_input)

print(sample_output.shape) 

(64, 50, 8000)
